In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

In [26]:
import json

In [27]:
# read data in to a list
data_list = []
# open file and read data in to the array
with open("./collected_data/data_all.txt") as f:
    for line in f:
        # print(line) # we inserted newline separator after each message received in the server
        try:
            x = json.loads(line)
        except ValueError:
            print("{0} was not proper json".format(line))
        except:
            print("Unexpected Error")
        else:
            # https://www.tutorialspoint.com/python/python_exceptions.htm
            # print(x) # if you want to print the data side-by-side
            data_list.append(x)

Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json
Socket Opened
 was not proper json

 was not proper jso

In [28]:
data = np.array(data_list)

In [29]:
data.shape

(24,)

In [30]:
#data[0]

In [31]:
articles = {"1": [], "2": [], "3": [], "4": [], "5": [] }

In [32]:
def clean_data(article_data):
    """
    retains 
    """
    new_data = []
    N = len(article_data)
    first_seen = article_data[0]
    for idx in range(0, N):
        curr = article_data[idx]
        if curr[0] != first_seen[0]:
            prev = article_data[idx - 1]
            if prev[1] != first_seen[1]:
                new_data.append([prev[0], int(prev[1]) - int(first_seen[1])])
                #print("{}, {}".format(int(prev[1]), int(first_seen[1])))
                first_seen = article_data[idx]
    return new_data

In [33]:
for item in data:
    article_num = item["article"]
    article_data = item["data"]
    if article_data:
        cleaned_article_data = clean_data(article_data)
        articles[article_num].append(cleaned_article_data)

In [34]:
print(len(articles["1"])+len(articles["2"])+len(articles["3"])+len(articles["4"])+len(articles["5"]))

21


In [35]:
#articles["2"][0]
#len(articles["2"])

In [36]:
hid_articles = []
vis_articles = []
for key in ["1", "2", "3", "4", "5"]:
    for arr in articles[key]:
        if key in ["1", "4"]:
            hid_articles.append(arr)
        else:
            vis_articles.append(arr)

In [37]:
print("{} {}".format(len(hid_articles), len(vis_articles)))

7 14


In [38]:
# for each article, get num of lines
num_lines = {"1": 0, "2": 0, "3": 0, "4": 0, "5": 0 }
for key in ["1", "2", "3", "4", "5"]:
    arr = articles[key][0]
    highest_line_num = max(arr, key=lambda x: x[0])
    num_lines[key] = highest_line_num[0]

In [39]:
# calculate average ms/line

avg_ms_line = {"1": 0, "2": 0, "3": 0, "4": 0, "5": 0}
for key in ["1", "2", "3", "4", "5"]:
    for arr in articles[key]:
        for pair in arr:
            avg_ms_line[key] += int(pair[1])
print(avg_ms_line)     # total ms spent on each

for key in ["1", "2", "3", "4", "5"]:
    n = len(articles[key])
    tot_num_lines = n * num_lines[key]
    avg_ms_line[key] /= tot_num_lines
print(avg_ms_line)     # ms/line for each article


avg_ms_hid = 0
avg_ms_vis = 0
total_lines = {"1": 0, "2": 0, "3": 0, "4": 0, "5": 0 }
for key in ["1", "2", "3", "4", "5"]:
    n = len(articles[key])
    avg = avg_ms_line[key]
    tot_num_lines = n * num_lines[key]
    total_lines[key] = tot_num_lines
    total_ms = avg * tot_num_lines
    if key in ["1", "4"]:
        avg_ms_hid += total_ms
    else:
        avg_ms_vis += total_ms
avg_ms_hid /= (total_lines["1"] + total_lines["4"])
avg_ms_vis /= (total_lines["2"] + total_lines["3"] + total_lines["5"])
print(avg_ms_hid, avg_ms_vis)

{'1': 246731, '2': 360882, '3': 441551, '4': 379271, '5': 401689}
{'1': 3575.8115942028985, '2': 3608.82, '3': 3839.573913043478, '4': 4309.897727272727, '5': 4016.89}
3987.2738853503183 3822.609523809524


In [40]:
# calculate reversions

penalties = {"1": 0, "2": 0, "3": 0, "4": 0, "5": 0 }
for key in ["1", "2", "3", "4", "5"]:
    for arr in articles[key]:
        for idx, pair in enumerate(arr[1:]):
            if pair[0] < arr[idx-1][0]:
                penalties[key] -= 1
            elif pair[0] > arr[idx-1][0] + 10:
                penalties[key] -= 1
print(penalties) # raw numbers

for key in ["1", "2", "3", "4", "5"]:
    penalties[key] /= total_lines[key]
print(penalties) # reversions per line

{'1': -95, '2': -146, '3': -160, '4': -111, '5': -91}
{'1': -1.3768115942028984, '2': -1.46, '3': -1.391304347826087, '4': -1.2613636363636365, '5': -0.91}
